In [1]:
#Imports
import numpy as np
import pandas as pd
from os.path import join
from datetime import datetime
from pathlib import Path

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from pytorch_lightning import loggers as pl_loggers

import pytorch_lightning as pl
from pytorch_lightning import Trainer, seed_everything, LightningModule
from pytorch_lightning.callbacks import ModelCheckpoint

from transformers import BertModel
from utils.data import RelevantDataset
import importlib

#Notebook Parameters
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
    
bert_id = "google/bert_uncased_L-2_H-128_A-2"

noetbook_datetime = datetime.now()

seed_everything(1337)

Global seed set to 1337


1337

# Training

## Set Architecture

In [2]:
from utils.architectures import RelevantModuleV03 as RelevantModule


## Load Datasets

In [3]:
train_ds = RelevantDataset(dataset="train", target_mode="isrelevant", device=device)
validation_ds = RelevantDataset(dataset="val", device = device, target_mode="isrelevant", dimensions = train_ds.dimensions)

## Hyperparameters

In [4]:
start_lr = 1e-4
batch_size = 16

In [5]:
# model
model = RelevantModule(
    bert = BertModel.from_pretrained(bert_id).to(device),
    input_size = sum(train_ds.dimensions[0][1]), 
    output_size = train_ds.dimensions[1],
    start_lr=start_lr,
    prior = train_ds.prior
)


#monitoring
architecture_name = model.__class__.__name__
logdir = join("logs", architecture_name, noetbook_datetime.strftime("%Y-%m-%dT%H-%M-%S"))
print(f"Logging to {logdir}")
Path(logdir).mkdir(parents=True, exist_ok=True)

tb_logger = pl_loggers.TensorBoardLogger(logdir, name="", version="")

checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    dirpath=logdir,
    verbose=True,
    save_last=True,
    save_top_k=-1, #save all
    mode="min",
    filename='-{epoch:02d}-{val_loss:.2f}'
)

#Dataloaders
train_dl = DataLoader(train_ds,batch_size  = batch_size, shuffle=True)
validation_dl = DataLoader(validation_ds, batch_size  = 64, shuffle=False)

# training
trainer = Trainer(
    gpus=1, 
    precision=16, 
    logger=tb_logger, 
    callbacks=[checkpoint_callback]
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
Using native 16bit precision.


Logging to logs\RelevantModuleV03\2021-05-04T12-25-12


## Executing run

In [ ]:
trainer.fit(model, train_dl, validation_dl)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type       | Params
-------------------------------------------------
0 | bert              | BertModel  | 4.4 M 
1 | linear_after_bert | Linear     | 33.0 K
2 | feed_forward      | Sequential | 714 K 
-------------------------------------------------
5.1 M     Trainable params
2         Non-trainable params
5.1 M     Total params
20.532    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Epoch 0, global step 16373: val_loss reached 0.09340 (best 0.09340), saving model to "E:\CHEERS_Challenge\CHEERS_challenge_round_1\logs\RelevantModuleV03\2021-05-04T12-25-12\-epoch=00-val_loss=0.09.ckpt" as top 1


Validating: 0it [00:00, ?it/s]

Epoch 1, global step 32747: val_loss reached 0.09107 (best 0.09107), saving model to "E:\CHEERS_Challenge\CHEERS_challenge_round_1\logs\RelevantModuleV03\2021-05-04T12-25-12\-epoch=01-val_loss=0.09.ckpt" as top 2


# Notes

Visit: https://pytorch-lightning.readthedocs.io/en/latest/common/weights_loading.html#restoring-training-state

In [ ]:
#import utils.data
#utils.data.preprocess("data_round_1")

In [14]:
from torch.nn import functional as F

In [17]:
F.one_hot(torch.tensor([1,0]), num_classes = 2)

tensor([[0, 1],
        [1, 0]])